In [1]:
import pandas as pd
import warnings
from elasticsearch import Elasticsearch
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import ElasticsearchStore
import nltk
from nltk.tokenize import sent_tokenize
from langchain.text_splitter import CharacterTextSplitter

In [2]:
# warning blind
warnings.filterwarnings('ignore') #default : warning 다시 보이게 함

In [3]:
df = pd.read_csv('TEXT_UTF_20240222.csv', encoding='utf-8')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   DOC_ID      250 non-null    int64 
 1   DOC_TITLE   250 non-null    object
 2   Unnamed: 2  209 non-null    object
 3   META        250 non-null    object
 4   HEADER      250 non-null    object
 5   TEXT        250 non-null    object
dtypes: int64(1), object(5)
memory usage: 11.8+ KB


In [5]:
df.head()

,DOC_ID,DOC_TITLE,Unnamed: 2,META,HEADER,TEXT
0,1,교육과정_편성_및_운영지침(20230828)_데모_작업,NaN,제1조(목적),제1조(목적),이 지침은 학칙시행세칙 제3장에서 정한 교육과정 관련 규정에 대한 세부사항을 정함을...
1,1,교육과정_편성_및_운영지침(20230829)_데모_작업,NaN,제2조(편성원칙),제2조(편성원칙),①교육과정은 사회 변화와 학생 요구 등을 반영하고 본교 교육목표에 맞는 인재를 양성...
2,1,교육과정_편성_및_운영지침(20230830)_데모_작업,NaN,제3조(교과목의 구분),제3조(교과목의 구분),①학사과정의 교과목은 다음 각호와 같이 구분한다.\r\n1. 성균교양(중점·균형)과...
3,1,교육과정_편성_및_운영지침(20230831)_데모_작업,NaN,제4조(교과목 편성시기),제4조(교과목 편성시기),①교과목은 매학년도 1회 교무처에서 안내하는 지정기간내에 편성하는\r\n것을 원칙으...
4,1,교육과정_편성_및_운영지침(20230832)_데모_작업,NaN,제5조(교과목 편성신청 및 승인),제5조(교과목 편성신청 및 승인),"①교과목의 편성(신규 및 변경, 삭제 포함)은 학과장의 요청에 따라 학장이 공문으로..."


In [6]:
df['HEADER'][0]

' 제1조(목적)'

In [7]:
for i in range(len(df)):
    text = df['HEADER'][i]
    df['HEADER'][i] = text.strip()

In [8]:
df['HEADER'][0]

'제1조(목적)'

In [9]:
df[20:25]

,DOC_ID,DOC_TITLE,Unnamed: 2,META,HEADER,TEXT
20,2,성균관대학교_학칙_본문,제 1 장 총 칙,제3조(정의),제 1 장 총 칙 제3조(정의),"①“학사과정”이라 함은 학사학위를, “석사과정”이라 함은 석사학위를, “학석사통합과..."
21,2,성균관대학교_학칙_본문,제 2 장 편제 제 1 절 교육조직,제4조(교육조직),제 2 장 편제 제 1 절 교육조직 제4조(교육조직),"①본교에 다음과 같은 대학을 둔다. 1. 학부대학 2. 유학대학, 문과대학, 사회과..."
22,2,성균관대학교_학칙_본문,제 2 장 편제 제 1 절 교육조직,제5조(과정),제 2 장 편제 제 1 절 교육조직 제5조(과정),"①대학에 학사과정(이하 ‘학석사통합과정 내 학사과정’을 포함한다.)을 두며, 학위를..."
23,2,성균관대학교_학칙_본문,제 2 장 편제 제 1 절 교육조직,제6조(과정별 학과 편성 및 입학정원),제 2 장 편제 제 1 절 교육조직 제6조(과정별 학과 편성 및 입학정원),①학사과정의 대학에 설치하는 학과(학부)(이하 ‘학과’라 한다)와 모집단위별 입학정...
24,2,성균관대학교_학칙_본문,제 2 장 편제 제 1 절 교육조직,제6조의2(학사과정 및 대학원과정의 전공 이수 방법),제 2 장 편제 제 1 절 교육조직 제6조의2(학사과정 및 대학원과정의 전공 이수 방법),"①학사과정 학생은 소속학과를 제1전공으로 이수하여야 한다. 다만, 총장의 허가를 받..."


In [10]:
import re

def textReplace(str):
    result = re.sub('\r\n', ' ', str)
    return result

In [11]:
df['NEW_BODY'] = None

In [12]:
for i in range(len(df)):
    body = df['TEXT'][i]

    new_body = textReplace(body)
    df['NEW_BODY'][i] = new_body

# VECTORDB CONNECT

In [13]:
ELASTIC_SEARCH_SERVER = "https://user_elasticsearch_ip:9200"
ELASTIC_USER = "userid"
ELASTIC_PASSWORD = "password"

In [14]:
# Connect Elasticsearch server
es_connection = Elasticsearch(ELASTIC_SEARCH_SERVER,
                            ca_certs=False,
                            verify_certs=False,
                            basic_auth =( ELASTIC_USER , ELASTIC_PASSWORD ),
                            timeout=120)

es_connection.info()

ObjectApiResponse({'name': 'es01', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'BYTTI105SDiW5TWzkXd2Sw', 'version': {'number': '8.12.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '6185ba65d27469afabc9bc951cded6c17c21e3f3', 'build_date': '2024-02-01T13:07:13.727175297Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [15]:
# index config
index_body = {
  "settings": {
      "number_of_shards": 1,
      "number_of_replicas": 0
  },  
  "mappings": {
    "properties": {
      "metadata": {
        "properties": {
          "doc_id": {
            "type": "text",
            "fields": {
              "keyword": {
                "type": "keyword",
                "ignore_above": 256
              }
            }
          },
        "table_id": {
            "type": "text",
            "fields": {
              "keyword": {
                "type": "keyword",
                "ignore_above": 256
              }
            }
          },
          "section": {
            "type": "text"
          },
          "seq": {
            "type": "integer"
          },
          "chunk_num": {
            "type": "integer"
          },
          "doc_name": {
            "type": "text"
          }
        }
      },
      "text": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 8191
          }
        }
      },
      "vector": {
      "type": "dense_vector",
      "dims": 1024,
      "index": 'true',
      "similarity": "cosine"
      }
    }
  }
}

In [16]:
# index name input
index_id = 'skku_gpt_240227'

In [17]:
response = es_connection.indices.create(index=index_id, body=index_body)
print(response)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'skku_gpt_240227'}


In [18]:
# model name input
embd_model_id = "intfloat/multilingual-e5-large"
embd_model = HuggingFaceEmbeddings(model_name=embd_model_id)

In [19]:
index_id = 'skku_gpt_240227'

db = ElasticsearchStore(
    embedding=embd_model,            #embedding model
    es_connection=es_connection,     #existing Elasticsearch instance
    index_name=index_id,             #index(=table) name
)

db.client.indices.refresh(index=index_id)

ObjectApiResponse({'_shards': {'total': 1, 'successful': 1, 'failed': 0}})

# For text chunking

In [20]:
def ko_text_chunk(body, chunk_size):
    text_list = sent_tokenize(body)
    chunks = []
    current_chunk = []
    current_length = 0

    for text in text_list:        
        if current_length + len(text) <= chunk_size:
            current_chunk.append(text)
            current_length += len(text)
        else:
            chunk = ' '.join(current_chunk)
            chunks.append(chunk)
            current_chunk = [text]
            current_length = len(text)

    if current_chunk:
        chunk = ' '.join(current_chunk)
        chunks.append(chunk)
    
    return chunks

In [21]:
def page_ko_to_docs( chunk_size, body, doc_id, doc_nm, section, header):    
    # list to chunk
    chunks = ko_text_chunk(body, chunk_size)
    # chunks = ko_text_chunk_overlap(body, chunk_size, overlap_size)
    # list(str) to list(document)
    text_splitter = CharacterTextSplitter()
    txt_docs = text_splitter.create_documents(chunks)
    
    chunk_num = 0
    for doc in txt_docs:
        chunk_num += 1
        doc.page_content = header + " " + doc.page_content
        doc.metadata = {"doc_id":doc_id, "chunk_num":chunk_num, "doc_name":doc_nm, "section": section}
        
    return txt_docs

In [22]:
df.head(2)

,DOC_ID,DOC_TITLE,Unnamed: 2,META,HEADER,TEXT,NEW_BODY
0,1,교육과정_편성_및_운영지침(20230828)_데모_작업,NaN,제1조(목적),제1조(목적),이 지침은 학칙시행세칙 제3장에서 정한 교육과정 관련 규정에 대한 세부사항을 정함을...,이 지침은 학칙시행세칙 제3장에서 정한 교육과정 관련 규정에 대한 세부사항을 정함을...
1,1,교육과정_편성_및_운영지침(20230829)_데모_작업,NaN,제2조(편성원칙),제2조(편성원칙),①교육과정은 사회 변화와 학생 요구 등을 반영하고 본교 교육목표에 맞는 인재를 양성...,①교육과정은 사회 변화와 학생 요구 등을 반영하고 본교 교육목표에 맞는 인재를 양성...


In [23]:
# ko
chunk_size = 5120
page_ko_total_docs = []

for row in df.iterrows():
    # create metadata

    doc_id = row[1]['DOC_ID']
    doc_name = row[1]['DOC_TITLE'] if row[1]['DOC_TITLE'] is not None else ''
    section = row[1]['META']
    header = row[1]['HEADER']
        # create content
    text = row[1]['NEW_BODY'] if row[1]['NEW_BODY'] is not None else ''
   
    # create documents
    docs = page_ko_to_docs(chunk_size, text, doc_id, doc_name, section, header)
    
    # docs to list
    page_ko_total_docs.append(docs) 

In [24]:
len(page_ko_total_docs)

250

In [25]:
page_ko_total_docs[0]

[Document(page_content='제1조(목적) 이 지침은 학칙시행세칙 제3장에서 정한 교육과정 관련 규정에 대한 세부사항을 정함을 목적으로 한다.', metadata={'doc_id': 1, 'chunk_num': 1, 'doc_name': '교육과정_편성_및_운영지침(20230828)_데모_작업', 'section': '제1조(목적)'})]

In [26]:
page_ko_total_docs[20]

[Document(page_content='제 1 장 총 칙 제3조(정의) ①“학사과정”이라 함은 학사학위를, “석사과정”이라 함은 석사학위를, “학석사통합과정”이라 함은 학사학위와 석사학위를, “박사과정”이라 함은 박사학위를, “석박사통합과정”이라 함은 학사에게 박사학위를 각각 수여하기 위한 과정을 말하며, “복합학위과정”이라 함은 의무전문석사학위(M.D. )와 의학학술박사학위(Ph.D.)를 동시에 수여하기 위한 과정을 말한다.', metadata={'doc_id': 2, 'chunk_num': 1, 'doc_name': '성균관대학교_학칙_본문', 'section': '제3조(정의)'})]

# Index to ES

In [27]:
j = 0
for i in range(len(page_ko_total_docs)):
    db.add_documents(
    page_ko_total_docs[i],
    embedding=embd_model, 
    index_name=index_id,             #index(=table) name
    vector_query_field="vector" #mapping vector field name
)
    j = j + 1
    
print(j)

250


# Index 부칙

In [28]:
df2 = pd.read_csv('부칙_UTF.csv', encoding='utf-8')

In [29]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   DOC_ID     5 non-null      int64 
 1   DOC_TITLE  5 non-null      object
 2   BODY       5 non-null      object
dtypes: int64(1), object(2)
memory usage: 248.0+ bytes


In [30]:
df2.head()

,DOC_ID,DOC_TITLE,BODY
0,1,교육과정 편성 및 운영지침,부칙\r\n이 지침은 2018년 3월 1일부터 시행한다.\r\n부칙\r\n이 지침은...
1,2,성균관대학교_학칙_본문,부칙 본 학칙은 1955년 4월 1일부터 시행한다. 부칙(1976년 3월 24일 개...
2,4,성균관대학교학칙시행세칙_본문,부칙 ①(시행일) 이 세칙은 2019년 8월 31일부터 시행한다. ②(다른 세칙의 ...
3,6,휴학복학처리에관한지침,부칙 이 개정 지침은 2018년 12월 1일부터 시행한다. 부칙 제1조(시행일) 이...
4,7,수업운영규정,부칙 ①(시행일) 이 규정은 2012년 3월 1일부터 시행한다. ②(다른 규정의 폐...


In [31]:
df2['NEW_BODY'] = None

In [32]:
for i in range(len(df2)):
    body = df2['BODY'][i]

    new_body = textReplace(body)
    df2['NEW_BODY'][i] = new_body

In [33]:
df2.head()

,DOC_ID,DOC_TITLE,BODY,NEW_BODY
0,1,교육과정 편성 및 운영지침,부칙\r\n이 지침은 2018년 3월 1일부터 시행한다.\r\n부칙\r\n이 지침은...,부칙 이 지침은 2018년 3월 1일부터 시행한다. 부칙 이 지침은 2021년 2월...
1,2,성균관대학교_학칙_본문,부칙 본 학칙은 1955년 4월 1일부터 시행한다. 부칙(1976년 3월 24일 개...,부칙 본 학칙은 1955년 4월 1일부터 시행한다. 부칙(1976년 3월 24일 개...
2,4,성균관대학교학칙시행세칙_본문,부칙 ①(시행일) 이 세칙은 2019년 8월 31일부터 시행한다. ②(다른 세칙의 ...,부칙 ①(시행일) 이 세칙은 2019년 8월 31일부터 시행한다. ②(다른 세칙의 ...
3,6,휴학복학처리에관한지침,부칙 이 개정 지침은 2018년 12월 1일부터 시행한다. 부칙 제1조(시행일) 이...,부칙 이 개정 지침은 2018년 12월 1일부터 시행한다. 부칙 제1조(시행일) 이...
4,7,수업운영규정,부칙 ①(시행일) 이 규정은 2012년 3월 1일부터 시행한다. ②(다른 규정의 폐...,부칙 ①(시행일) 이 규정은 2012년 3월 1일부터 시행한다. ②(다른 규정의 폐...


In [34]:
df2['LEN'] = None

In [35]:
for i in range(len(df2)):
    text = df2['NEW_BODY'][i]
    df2['LEN'][i] = len(text)

In [36]:
df2.head()

,DOC_ID,DOC_TITLE,BODY,NEW_BODY,LEN
0,1,교육과정 편성 및 운영지침,부칙\r\n이 지침은 2018년 3월 1일부터 시행한다.\r\n부칙\r\n이 지침은...,부칙 이 지침은 2018년 3월 1일부터 시행한다. 부칙 이 지침은 2021년 2월...,118
1,2,성균관대학교_학칙_본문,부칙 본 학칙은 1955년 4월 1일부터 시행한다. 부칙(1976년 3월 24일 개...,부칙 본 학칙은 1955년 4월 1일부터 시행한다. 부칙(1976년 3월 24일 개...,30779
2,4,성균관대학교학칙시행세칙_본문,부칙 ①(시행일) 이 세칙은 2019년 8월 31일부터 시행한다. ②(다른 세칙의 ...,부칙 ①(시행일) 이 세칙은 2019년 8월 31일부터 시행한다. ②(다른 세칙의 ...,2127
3,6,휴학복학처리에관한지침,부칙 이 개정 지침은 2018년 12월 1일부터 시행한다. 부칙 제1조(시행일) 이...,부칙 이 개정 지침은 2018년 12월 1일부터 시행한다. 부칙 제1조(시행일) 이...,258
4,7,수업운영규정,부칙 ①(시행일) 이 규정은 2012년 3월 1일부터 시행한다. ②(다른 규정의 폐...,부칙 ①(시행일) 이 규정은 2012년 3월 1일부터 시행한다. ②(다른 규정의 폐...,615


In [37]:
df2['NEW_BODY'][0]

'부칙 이 지침은 2018년 3월 1일부터 시행한다. 부칙 이 지침은 2021년 2월 22일부터 시행한다. 부칙 이 지침은 2022년 2월 21일부터 시행한다. 부칙 이 지침은 2023년 8월 28일부터 시행한다.'

In [38]:
df2['NEW_BODY'][0].split('부칙')

['',
 ' 이 지침은 2018년 3월 1일부터 시행한다. ',
 ' 이 지침은 2021년 2월 22일부터 시행한다. ',
 ' 이 지침은 2022년 2월 21일부터 시행한다. ',
 ' 이 지침은 2023년 8월 28일부터 시행한다.']

In [39]:
bu = df2['NEW_BODY'][0].split('부칙')

bu.pop(0)

''

In [40]:
bu

[' 이 지침은 2018년 3월 1일부터 시행한다. ',
 ' 이 지침은 2021년 2월 22일부터 시행한다. ',
 ' 이 지침은 2022년 2월 21일부터 시행한다. ',
 ' 이 지침은 2023년 8월 28일부터 시행한다.']

In [41]:
def page_ko_to_docs(body, doc_id, doc_nm, section):    
    # list to chunk
    # chunks = ko_text_chunk(body, chunk_size)
    # chunks = ko_text_chunk_overlap(body, chunk_size, overlap_size)
    # list(str) to list(document)
    text_splitter = CharacterTextSplitter()
    txt_docs = text_splitter.create_documents([body])
    
    chunk_num = 0
    for doc in txt_docs:
        chunk_num += 1
        doc.page_content = doc.page_content
        doc.metadata = {"doc_id":doc_id, "chunk_num":chunk_num, "doc_name":doc_nm, "section": section}
        
    return txt_docs

In [50]:
y= 0
bu = df2['NEW_BODY'][y].split('부칙')
bu.pop(0)

# ko
chunk_size = 10000
page_ko_total_docs = []

for i in range(len(bu)):
    # create metadata

    doc_id = df2['DOC_ID'][y]
    doc_name = df2['DOC_TITLE'][y]
    section = '부칙'
        # create content
    text = '부칙' + bu[i]
   
    # create documents
    docs = page_ko_to_docs(text, doc_id, doc_name, section)
    
    # docs to list
    page_ko_total_docs.append(docs) 

In [51]:
len(page_ko_total_docs)

4

In [52]:
page_ko_total_docs[0]

[Document(page_content='부칙 이 지침은 2018년 3월 1일부터 시행한다.', metadata={'doc_id': 1, 'chunk_num': 1, 'doc_name': '교육과정 편성 및 운영지침', 'section': '부칙'})]

In [53]:
page_ko_total_docs[3]

[Document(page_content='부칙 이 지침은 2023년 8월 28일부터 시행한다.', metadata={'doc_id': 1, 'chunk_num': 1, 'doc_name': '교육과정 편성 및 운영지침', 'section': '부칙'})]

In [54]:
j = 0
for i in range(len(page_ko_total_docs)):
    db.add_documents(
    page_ko_total_docs[i],
    embedding=embd_model, 
    index_name=index_id,             #index(=table) name
    vector_query_field="vector" #mapping vector field name
)
    j = j + 1
    
print(j)

4


In [55]:
y= 1
bu = df2['NEW_BODY'][y].split('부칙')

bu.pop(0)

# ko
chunk_size = 10000
page_ko_total_docs = []

for i in range(len(bu)):
    # create metadata

    doc_id = df2['DOC_ID'][y]
    doc_name = df2['DOC_TITLE'][y]
    section = '부칙'
        # create content
    text = '부칙' + bu[i]
   
    # create documents
    docs = page_ko_to_docs(text, doc_id, doc_name, section)
    
    # docs to list
    page_ko_total_docs.append(docs) 

In [56]:
len(page_ko_total_docs)

128

In [57]:
page_ko_total_docs[0]

[Document(page_content='부칙 본 학칙은 1955년 4월 1일부터 시행한다.', metadata={'doc_id': 2, 'chunk_num': 1, 'doc_name': '성균관대학교_학칙_본문', 'section': '부칙'})]

In [58]:
page_ko_total_docs[10]

[Document(page_content='부칙(1981년 6월 10일 개정) ①본 개정학칙은 1981년 6월 10일부터 시행한다. ②본 개정학칙은 1981년 제1학년 입학생부터 적용한다. ③본 개정학칙 시행당시의 제2부 재학생(2학년이상)은 주간부 교과과정을 준용하며 졸업학점은 학칙시행세칙(Ⅱ)의 제3조 규정에 의한다. ④1981학년도 본 개정학칙 시행당시의 농과대학 원예조경학과의 재학생(2학년이상)은 조경학과에 전과한 것으로 본다.', metadata={'doc_id': 2, 'chunk_num': 1, 'doc_name': '성균관대학교_학칙_본문', 'section': '부칙'})]

In [59]:
j = 0
for i in range(len(page_ko_total_docs)):
    db.add_documents(
    page_ko_total_docs[i],
    embedding=embd_model, 
    index_name=index_id,             #index(=table) name
    vector_query_field="vector" #mapping vector field name
)
    j = j + 1
    
print(j)

128


In [60]:
y= 2
bu = df2['NEW_BODY'][y].split('부칙')

bu.pop(0)

# ko
chunk_size = 10000
page_ko_total_docs = []

for i in range(len(bu)):
    # create metadata

    doc_id = df2['DOC_ID'][y]
    doc_name = df2['DOC_TITLE'][y]
    section = '부칙'
        # create content
    text = '부칙' + bu[i]
   
    # create documents
    docs = page_ko_to_docs(text, doc_id, doc_name, section)
    
    # docs to list
    page_ko_total_docs.append(docs) 

In [61]:
j = 0
for i in range(len(page_ko_total_docs)):
    db.add_documents(
    page_ko_total_docs[i],
    embedding=embd_model, 
    index_name=index_id,             #index(=table) name
    vector_query_field="vector" #mapping vector field name
)
    j = j + 1
    
print(j)

20


In [62]:
y= 3
bu = df2['NEW_BODY'][y].split('부칙')

bu.pop(0)

# ko
chunk_size = 10000
page_ko_total_docs = []

for i in range(len(bu)):
    # create metadata

    doc_id = df2['DOC_ID'][y]
    doc_name = df2['DOC_TITLE'][y]
    section = '부칙'
        # create content
    text = '부칙' + bu[i]
   
    # create documents
    docs = page_ko_to_docs(text, doc_id, doc_name, section)
    
    # docs to list
    page_ko_total_docs.append(docs) 

In [63]:
j = 0
for i in range(len(page_ko_total_docs)):
    db.add_documents(
    page_ko_total_docs[i],
    embedding=embd_model, 
    index_name=index_id,             #index(=table) name
    vector_query_field="vector" #mapping vector field name
)
    j = j + 1
    
print(j)

3


In [64]:
y= 4
bu = df2['NEW_BODY'][y].split('부칙')

bu.pop(0)

# ko
chunk_size = 10000
page_ko_total_docs = []

for i in range(len(bu)):
    # create metadata

    doc_id = df2['DOC_ID'][y]
    doc_name = df2['DOC_TITLE'][y]
    section = '부칙'
        # create content
    text = '부칙' + bu[i]
   
    # create documents
    docs = page_ko_to_docs(text, doc_id, doc_name, section)
    
    # docs to list
    page_ko_total_docs.append(docs) 

In [65]:
j = 0
for i in range(len(page_ko_total_docs)):
    db.add_documents(
    page_ko_total_docs[i],
    embedding=embd_model, 
    index_name=index_id,             #index(=table) name
    vector_query_field="vector" #mapping vector field name
)
    j = j + 1
    
print(j)

14


# change 부칙 for hybrid index

In [66]:
index_id = 'skku_hybrid_240226'

db = ElasticsearchStore(
    embedding=embd_model,            #embedding model
    es_connection=es_connection,     #existing Elasticsearch instance
    index_name=index_id,             #index(=table) name
)

db.client.indices.refresh(index=index_id)

ObjectApiResponse({'_shards': {'total': 1, 'successful': 1, 'failed': 0}})

In [67]:
df2

,DOC_ID,DOC_TITLE,BODY,NEW_BODY,LEN
0,1,교육과정 편성 및 운영지침,부칙\r\n이 지침은 2018년 3월 1일부터 시행한다.\r\n부칙\r\n이 지침은...,부칙 이 지침은 2018년 3월 1일부터 시행한다. 부칙 이 지침은 2021년 2월...,118
1,2,성균관대학교_학칙_본문,부칙 본 학칙은 1955년 4월 1일부터 시행한다. 부칙(1976년 3월 24일 개...,부칙 본 학칙은 1955년 4월 1일부터 시행한다. 부칙(1976년 3월 24일 개...,30779
2,4,성균관대학교학칙시행세칙_본문,부칙 ①(시행일) 이 세칙은 2019년 8월 31일부터 시행한다. ②(다른 세칙의 ...,부칙 ①(시행일) 이 세칙은 2019년 8월 31일부터 시행한다. ②(다른 세칙의 ...,2127
3,6,휴학복학처리에관한지침,부칙 이 개정 지침은 2018년 12월 1일부터 시행한다. 부칙 제1조(시행일) 이...,부칙 이 개정 지침은 2018년 12월 1일부터 시행한다. 부칙 제1조(시행일) 이...,258
4,7,수업운영규정,부칙 ①(시행일) 이 규정은 2012년 3월 1일부터 시행한다. ②(다른 규정의 폐...,부칙 ①(시행일) 이 규정은 2012년 3월 1일부터 시행한다. ②(다른 규정의 폐...,615


In [68]:
df2['NEW_BODY'][4]

'부칙 ①(시행일) 이 규정은 2012년 3월 1일부터 시행한다. ②(다른 규정의 폐지) 이 규정 시행과 동시에 「수업운영에 관한 내규」는 폐지한다. 부칙 ①(시행일) 이 규정은 2012년 10월 1일부터 시행하며 2013학년도 이후 수업편성 및 운영에 적용한다. ②(경과조치) 2012학년도 수업 편성 및 운영은 종전 규정을 적용한다. 부칙 ①(시행일) 이 규정은 2013년 11월 1일부터 시행하며 2014학년도 이후 수업편성 및 운영에 적용한다. ②(경과조치) 2013학년도 수업 편성 및 운영은 종전 규정을 적용한다. 부칙 이 규정은 2015년 3월 1일부터 시행한다. 부칙 이 규정은 2016년 1월 1일부터 시행한다. 부칙 이 규정은 2016년 3월 1일부터 시행한다. 부칙 이 규정은 2016년 11월 1일부터 적용한다. 부칙 이 규정은 2018년 3월 1일부터 적용한다. 부칙 이 규정은 2018년 7월 1일부터 적용한다. 부칙 이 규정은 2019년 3월 1일부터 적용한다. 부칙 이 규정은 2020년 3월 1일부터 적용한다. 부칙 이 규정은 2021년 2월 22일부터 시행한다. 부칙 이 규정은 2021년 2월 22일부터 시행한다. 부칙 이 규정은 2023년 2월 27일부터 시행한다.'

In [69]:
# ko
chunk_size = 512
page_ko_total_docs = []

for y in range(len(df2)):
    bu = df2['NEW_BODY'][y].split('부칙')
    bu.pop(0)

    
    for i in range(len(bu)):
    # create metadata
        doc_id = df2['DOC_ID'][y]
        doc_name = df2['DOC_TITLE'][y]
        section = '부칙'
            # create content
        text = '부칙' + bu[i]
       
        # create documents
        docs = page_ko_to_docs(text, doc_id, doc_name, section)
        
        # docs to list
        page_ko_total_docs.append(docs) 

In [70]:
len(page_ko_total_docs)

169

In [71]:
page_ko_total_docs[0]

[Document(page_content='부칙 이 지침은 2018년 3월 1일부터 시행한다.', metadata={'doc_id': 1, 'chunk_num': 1, 'doc_name': '교육과정 편성 및 운영지침', 'section': '부칙'})]

In [74]:
page_ko_total_docs[3]

[Document(page_content='부칙 이 지침은 2023년 8월 28일부터 시행한다.', metadata={'doc_id': 1, 'chunk_num': 1, 'doc_name': '교육과정 편성 및 운영지침', 'section': '부칙'})]

In [73]:
page_ko_total_docs[4]

[Document(page_content='부칙 본 학칙은 1955년 4월 1일부터 시행한다.', metadata={'doc_id': 2, 'chunk_num': 1, 'doc_name': '성균관대학교_학칙_본문', 'section': '부칙'})]

In [72]:
page_ko_total_docs[5]

[Document(page_content='부칙(1976년 3월 24일 개정) (학무 1010-469) ①본 개정학칙은 1976년 3월 1일부터 시행한다. ②본 개정학칙은 1976학년도 제1학년 입학생부터 적용한다. ③1975학년도 이전 입학생에 대하여는 종전의 학칙과 본 개정학칙 제45조 내지 제48조의 규정을 적용한다. ④1975학년도 이전 입학생이 실험대학 해당 학년에 복학 또는 재입학하는 경우에는 따로 정하는 규정을 적용한다. ⑤본 개정학칙의 시행과 동시에 제2부 학칙은 폐지한다.', metadata={'doc_id': 2, 'chunk_num': 1, 'doc_name': '성균관대학교_학칙_본문', 'section': '부칙'})]

In [75]:
j = 0
for i in range(len(page_ko_total_docs)):
    db.add_documents(
    page_ko_total_docs[i],
    embedding=embd_model, 
    index_name=index_id,             #index(=table) name
    vector_query_field="vector" #mapping vector field name
)
    j = j + 1
    
print(j)

169
